---
## Ch.2　ニューラルネットの基礎

---
### 2.1　教師あり学習
入力変数ベクトル $\mathbf{x} \in \mathcal{X}$、出力変数 $y \in \mathcal{Y}$ について  
ある変数 $\mathbf{x}$ が与えられた時に $y$ を予測するモデルを学習することを目的とする。

教師あり学習の定義
$|\mathcal{D}|$ 個の訓練事例を $\mathcal{D} = \{(\mathbf{x}^{(n)}, y^{(n)})\}_{n=1}^{|\mathcal{D}|}$ としたとき、  
訓練データでの誤差（以下の目的関数）を最小化するモデルパラメータ $\mathbf{\theta}$ の値を求める手続きを教師あり学習と呼ぶ。

\begin{align*}
L(\mathbf{\theta}) = \frac{1}{|\mathcal{D}|} \sum_{n=1}^{|\mathcal{D}|} l_{\mathbf{\theta}} (\mathbf{x}^{(n)}, y^{(n)})
\end{align*}

自然言語処理では出力は単語のように離散集合 $\mathcal{Y} = \{1, \ldots, |\mathcal{Y}| \}$ であることが多く、分類問題として定式化できる。  
分類問題では 0-1損失関数の代理損失（surrogate loss）として、  
次の交差エントロピー損失関数と（多クラス）ヒンジ損失関数がよく用いられる。

・cross-entropy loss

\begin{align*}
l_{\mathbf{\theta}}^{\text{cross-entropy}} \left( \mathbf{x}^{(n)}, y^{(n)} \right) = - \log \frac{\exp ( f_{\mathbf{\theta}}( \mathbf{x}^{(n)}, y^{(n)} ))}{\sum_{\tilde{y} \in \mathcal{Y}} \exp ( f_{\mathbf{\theta}}(\mathbf{x}^{(n)}, \tilde{y}))}
\end{align*}

・multiclass hinge loss

\begin{align*}
l_{\theta}^{\text{hinge}}\left( \mathbf{x}^{(n)}, y^{(n)} \right) = \max \left( 0, \, 1 - f_{\theta}\left( \mathbf{x}^{(n)}, y^{(n)} \right) + \max_{\tilde{y} \in {\mathcal{Y} \\ \{y^{(n)}\}} } f_{\theta}( \mathbf{x}^{(n)}, \tilde{y}) \right)
\end{align*}

---
$f_{\theta}(\mathbf{x},y)$ は $\theta$ をパラメータとするスコア関数で、$\hat{y} = \operatorname{argmax}_{y} f_{\theta}(\mathbf{x}^{(n)}, y)$ を予測として使用することを想定している。  
注記なき限りスコア関数はニューラルネットによる実装とする。

誤差関数として交差エントロピー、スコア関数 $f_{\theta}(\mathbf{x},y)$ を線形モデルとするとき多項ロジスティック回帰という。  
交差エントロピー損失関数は以下の条件付き確率モデルを想定し、  
その負の対数尤度を想定していると考えることもできる。  

\begin{align*}
P_{\theta}(y|\mathbf{x}) = \frac{\exp ( f_{\theta} (\mathbf{x},y))}{\sum_{\tilde{y} \in \mathcal{Y}} \exp ( f_{\theta} ( \mathbf{x}, \tilde{y}))}
\end{align*}

交差エントロピー損失関数は、真の分布 $P^{*}_{\theta}(y|\mathbf{x})$ とモデル $P_{\theta}(y|\mathbf{x})$ との距離を表す交差エントロピーを  
訓練データで経験近似していることに相当することからそう呼ばれる。

ソフトマックス関数

\begin{align*}
\operatorname{softmax}(\mathbf{o})_{y} = \frac{\exp (o_{y})}{\sum_{\tilde{y} \in \mathcal{Y}} \exp (o_{y})}
\end{align*}

を用いると、上の式は

\begin{align*}
P_{\theta}(y|\mathbf{x}) = \operatorname{softmax}(\mathbf{o})_{y}
\end{align*}

と書くことができる。

---
### 2.2　順伝播型ニューラルネット
非線形変換を $L$ 回繰り返すFFNN (feedforward neural network) を想定する。  
$l$ 回目の変換結果の出力ベクトル $\mathbf{h}^{(l)} \in \mathcal{R}^{N^{(1)}}$ を隠れ状態ベクトルと呼ぶ。  

出力層からの出力候補 $y$ に対するスコアの計算は $|\mathcal{Y}| \times N^{(L)}$ のパラメータ行列 $\mathbf{W}^{(o)}$ と  
バイアスパラメータベクトル $\mathbf{b}^{(o)}$ を使って以下のように表す。

\begin{align*}
\mathbf{o} = \mathbf{W}^{(o)}\mathbf{h}^{(L)} + \mathbf{b}^{(o)}
\end{align*}

次に、各層の隠れ状態ベクトル $\mathbf{h}^{(l)} \, (l=1,\ldots,L)$ は活性化関数を $a$ として

\begin{align*}
\mathbf{h}^{(l)} = a^{(l)} \left( \mathbf{W}^{(l)}\mathbf{h}^{(l-1)} + \mathbf{b}^{(l)} \right)
\end{align*}

と表される。ここで $l=0$ のとき $\mathbf{h}^{0} = \mathbf{x}$ （$x$ は入力）とする。

---
### 2.3　活性化関数
シグモイド関数、tanh、ReLUなどがよく用いられる。  
活性化関数は非線形でなければならない。

---
### 2.4　勾配法
学習率を $\eta$ とするとき、次の更新式によって学習する。

\begin{align*}
\theta^{(k+1)} = \theta^{(k)} - \eta \partial L(\theta^{(k)})
\end{align*}

勾配法は局所的な最急降下方向に $\eta$ に比例してパラメータを変化させる。

バッチ法による勾配降下法を行うとき、  
損失関数の偏微分は個々のデータにおける偏微分の和に分解できる。

\begin{align*}
\partial L(\theta)^{batch} \frac{1}{|\mathcal{D}|} \sum_{n=1}^{|\mathcal{D}|} \partial l_{\theta}(\mathbf{x}^{(n)}, y^{(n)})
\end{align*}

バッチ法では訓練データ全てについての計算が必要なことから、  
ランダムに選んだ事例で近似する確率的勾配法がよく用いられる。  
１事例のみを用いるオンライン確率的勾配法では次のように近似する。$i$ はランダムに選んだ事例の番号。  
　（メモ：原文中の式に $i$ 見当たらず）

\begin{align*}
\partial L(\theta)^{\text{online}} = \partial l_{\theta} (\mathbf{x}^{(n)}, y^{(n)})
\end{align*}

ミニバッチ確率的勾配法では次の通り。

\begin{align*}
\partial L(\theta)^{\text{mini-batch}} = \frac{1}{|\mathcal{B}|} \sum_{m=1}^{|\mathcal{B}|} \partial l_{\theta} (\mathbf{x}^{(\mathcal{B}[m])}, y^{(\mathcal{B}[m])})
\end{align*}



---
### 2.5　誤差逆伝播法
(他のnotebookで取り扱ったため省略）

---
### 2.6　再帰ニューラルネット
Recurrent Neural Networksでは、長さ $T$ の入力ベクトル列 $\mathbf{X} = (\mathbf{x}_{1}, \mathbf{x}_{2}, \ldots, \mathbf{x}_{T_{x}})$ が与えられたとき、  
$l$ 層目の隠れ状態ベクトルを次のとおりに再帰的に更新する。

\begin{align*}
\mathbf{h}_{t}^{(l)} = a^{(l)} \left( \mathbf{W}^{(l)} \left[ \begin{array} \mathbf{h}_{t}^{(l-1)} \\\mathbf{h}_{t-1}^{(l)} \end{array} \right] + \mathbf{b}^{(l)} \right)
\end{align*}

RNNでは時刻 $t$ の $l-1$ 層目の隠れ状態ベクトルと時刻 $t-1$ の $l$ 層目の隠れ状態ベクトルを線形変換することから  
パラメータ行列 $\mathbf{W}^{(l)}$ の要素数は $(N^{(l)} + N^{(l-1)}) \times N^{(l)}$ となる。  
また、パラメータの初期値 $\mathbf{h}_{0}^{(l)}$ としては零ベクトルなど定数値が使われることが多い。

スコア関数は $\mathbf{h}_{t}^{(L)}$ を用いて次のように計算する。

\begin{align*}
f(\mathbf{x}, y) &= o_{t,y} \\
\mathbf{o}_{t} &= \mathbf{W}^{(o)} \mathbf{h}_{t}^{(L)} + \mathbf{b}^{(o)}
\end{align*}

RNNは時間方向に深いモデルとなっていること、時刻 $t$ によらない共通のパラメータを用いることが特徴である。  
誤差逆伝播法は時間鳳凰に遡ることからBPTTと呼ばれることがあるが、計算グラフ上は大きな違いがない。  


---
#### 双方向再帰ニューラルネット（bi-directional RNN）
時間の前向き方向と後ろ向き方向の走査を組み合わせた双方向RNNはよく用いられる。  
前向き走査の隠れ状態ベクトル $\overrightarrow{\mathbf{h}}$ と後ろ向き走査の隠れ状態ベクトル $\overleftarrow{\mathbf{h}}$ を次の式に従って計算する。

\begin{align*}
\overrightarrow{\mathbf{h}}_{t}^{(l)} &= a^{(l)} \left( \overrightarrow{W}^{(l)} \left[ \begin{array} \overrightarrow{\mathbf{h}}_{t}^{(l-1)} \\ \overleftarrow{\mathbf{h}}_{t}^{(l-1)} \\ \overrightarrow{\mathbf{h}}_{t-1}^{(l)} \end{array} \right] + \overrightarrow{\mathbf{b}}^{(l)} \right) \\
\overleftarrow{\mathbf{h}}_{t}^{(l)} &= a^{(l)} \left( \overleftarrow{W}^{(l)} \left[ \begin{array} \overrightarrow{\mathbf{h}}_{t}^{(l-1)} \\ \overleftarrow{\mathbf{h}}_{t}^{(l-1)} \\ \overrightarrow{\mathbf{h}}_{t+1}^{(l)} \end{array} \right] + \overleftarrow{\mathbf{b}}^{(l)} \right)
\end{align*}

出力 $\mathbf{o}_{t}$ については次の通り。

\begin{align*}
\mathbf{o}_{t} = \mathbf{W}^{(o)} \left[ \begin{array} \overrightarrow{\mathbf{h}}_{t}^{(L)} \\ \overleftarrow{\mathbf{h}}_{t}^{(L)} \end{array} \right] + \mathbf{b}^{(o)}
\end{align*}

上の式では各層で双方向の隠れ状態ベクトルを計算に用いるが  
この部分は手法により様々で、例えば逆向きの隠れ状態ベクトルを用いない計算式もある。

---
#### パラメータ行列の固有値による勾配不安定性の議論
パラメータ行列の固有値分解を次のように書く。

\begin{align*}
\mathbf{W} = \mathbf{Q} \mathbf{\Lambda} \mathbf{Q}^{-1}
\end{align*}

このときパラメータ行列の $t$ 乗は、$\mathbf{Q}^{-1}\mathbf{Q}$ が単位行列となることから次のようになる。

\begin{align*}
\mathbf{W}^{t} = \mathbf{Q} \mathbf{\Lambda}^{t} \mathbf{Q}^{-1}
\end{align*}

よって、RNNの隠れ状態ベクトルについて $\mathbf{h}_{t} = \mathbf{W}\mathbf{h}_{t-1}$ が成り立つとき、

\begin{align*}
\mathbf{h}_{t} &= \mathbf{W}\mathbf{h}_{t-1} = \mathbf{W}^{2}\mathbf{h}_{t-2} = \cdots = \mathbf{W}^{t}\mathbf{h}_{0} \\
&= \mathbf{Q} \mathbf{\Lambda}^{t} \mathbf{Q}^{-1} \mathbf{h}_{0}
\end{align*}

となることから、$t$ が大のとき固有値の値により状態変数の絶対値が強く影響されることとなる。  
ここから、パラメータ行列の固有値により勾配不安定性について議論することができる。  


---
### 2.7　ゲート付き再帰ニューラルネット
RNNは時間方向に深いため、長期の依存関係を学習するのが難しいという問題がある。  
ゲートはこれを解決する手法の一つである。  

ゲート $\mathbf{g}^{(f)}, \mathbf{g}^{(h)}$ を用いてショートカットを次の重み付き和へ拡張する。

\begin{align*}
\mathbf{h}^{(l)} = \mathbf{g}^{(f)} \odot f^{(l)}(\mathbf{h}^{(l-1)}) + \mathbf{g}^{(h)} \odot \mathbf{h}^{(l-1)}
\end{align*}

また、ゲート自体も学習可能な関数とする。

\begin{align*}
\mathbf{g}(\mathbf{h}) = a^{(g)}(\mathbf{W}^{(g)} \mathbf{h} + \mathbf{b}^{(g)})
\end{align*}


ゲート付RNNは時間方向の隠れ状態の計算にゲート付きショートカットを用いたRNNである。

まず、１つ前の時刻の隠れ状態ベクトル $\mathbf{h}_{t-1}$ へのショートカットを持ったRNNを考える。
再帰的な評価を行うと

\begin{align*}
\mathbf{h}_{t} &= f(\mathbf{h}_{t-1}) + \mathbf{h}_{t-1} = f(\mathbf{h}_{t-1}) + f(\mathbf{h}_{t-2}) + \mathbf{h}_{t-2} = \cdots \\
&= \sum_{i=1}^{t} f(\mathbf{h}_{i-1})
\end{align*}

これをゲート付きに拡張する。

\begin{align*}
\mathbf{h}_{t} &= f(\mathbf{h}_{t-1}) + \mathbf{g}_{t} \odot \mathbf{h}_{t-1} \\
&= f(\mathbf{h}_{t-1}) + \mathbf{g}_{t} \odot f(\mathbf{h}_{t-2}) + \mathbf{g}_{t} \odot \mathbf{g}_{t-1} \odot \mathbf{h}_{t-2} = \cdots \\
&= f(\mathbf{h}_{t-1}) + \sum_{i=1}^{t-1} ( \odot_{j=i}^{t} \mathbf{g}_{j}) f(\mathbf{h}_{i-1})
\end{align*}

ゲート自体を学習することで、忘却の度合いを調節することができるほか  
誤差が過去に直接伝播することで勾配消失が避けられる。

---
#### 長短期記憶（LSTM）
LSTM (long short-term memory) は最も代表的なゲート付RNNであり、  
入力ゲート $\mathbf{i}$、忘却ゲート $\mathbf{f}$、出力ゲート $\mathbf{o}$ を使用するほか  
セルと呼ばれる隠れ状態ベクトル $\mathbf{c}$ を追加する。

LSTMの一つは次のようなモデルとなる。

\begin{align*}
\left[ \begin{array} \bar{\mathbf{h}}_{t}^{(l)} \\ \mathbf{i}_{t}^{(l)} \\ \mathbf{f}_{t}^{(l)} \\ \mathbf{o}_{t}^{(l)} \end{array} \right] &= \left[ \begin{array} \text{tanh} \\ \text{sigmoid} \\ \text{sigmoid} \\ \text{sigmoid} \end{array} \right] \left( 
\left[ \begin{array} \mathbf{w}_{\bar{h}}^{(l)} \\ \mathbf{w}_{i}^{(l)} \\ \mathbf{w}_{f}^{(l)} \\ \mathbf{w}_{o}^{(l)} \end{array} \right]
\left[ \begin{array}\mathbf{h}_{t}^{(l-1)} \\ \mathbf{h}_{t-1}^{(l)} \end{array} \right]
 + \left[ \begin{array} \mathbf{b}_{\bar{h}}^{(l)} \\ \mathbf{b}_{i}^{(l)} \\ \mathbf{b}_{f}^{(l)} \\ \mathbf{b}_{o}^{(l)} \end{array} \right]
\right) \\
\mathbf{c}_{t}^{(l)} &= \mathbf{i}_{t}^{(l)} \odot \bar{h}_{t}^{(l)} + \mathbf{f}_{t}^{(l)} \odot \mathbf{c}_{t-1}^{(l)} \\
\mathbf{h}_{t}^{(l)} &= \mathbf{o}_{t}^{(l)} \odot \operatorname{tanh} ( \mathbf{c}_{t}^{(l)})
\end{align*}



---
#### ゲート付き回帰ユニット（GRU）
GRU (gated recurrent unit) はセルを使わないゲート付RNNで、  
再設定ゲート $\mathbf{r}$ と更新ゲート \mathbf{z}$ を用いて忘却と状態の更新を操作する。  

\begin{align*}
\left[ \begin{array} \mathbf{r}_{t}^{(l)} \\ \mathbf{z}_{t}^{(l)} \end{array} \right] &= \operatorname{sigmoid} \left( 
\left[ \begin{array} \mathbf{W}_{r}^{(l)} \\ \mathbf{W}_{z}^{(l)} \end{array} \right]
\left[ \begin{array}\mathbf{h}_{t}^{(l-1)} \\ \mathbf{h}_{t-1}^{(l)} \end{array} \right]
 + \left[ \begin{array} \mathbf{b}_{r}^{(l)} \\ \mathbf{b}_{z}^{(l)} \end{array} \right]
\right) \\
\tilde{\mathbf{h}}_{t}^{(l)} &= \operatorname{tanh} \left( \left[ \begin{array} \mathbf{h}_{t}^{(l-1)} \\ \mathbf{r}_{t}^{(l)} \odot \mathbf{h}_{t-1}^{(l)} \end{array} \right] \right) \\
\mathbf{h}_{t}^{(l)} &= (1-\mathbf{z}_{t}^{(l)}) \odot \tilde{\mathbf{h}}_{t}^{(l)} + \mathbf{z}_{t}^{(l)} \odot \mathbf{h}_{t-1}^{(l)}
\end{align*}


---
### 2.8　木構造再帰ニューラルネット
木構造再帰ニューラルネット（Recursive neural networks, Tree-RNN）はRNNを木構造に拡張したものである。

Tree-RNNでは木構造を事前に定義することが必要である。  
仮に二分木を使うと仮定し、隠れ状態ベクトル $\mathbf{h}_{P}$ の左の子と右の子を $\mathbf{h}_{L}, \mathbf{h}_{R}$ とすると

\begin{align*}
\mathbf{h}_{P} = a \left( \mathbf{W} \left[ \begin{array} \mathbf{h}_{L} \\ \mathbf{h}_{R} \end{array} \right] + \mathbf{b} \right)
\end{align*}

として $\mathbf{h}_{P}$ が計算される。

この演算は木構造の葉ノードからルートノードに向かって順次計算される。  
また、垂直方向に層を重ねるようにすることもできる。

\begin{align*}
\mathbf{h}_{P}^{(l)} = a \left( \mathbf{W}^{(l)} \left[ \begin{array} \mathbf{h}_{L}^{(l)} \\ \mathbf{h}_{R}^{(l)} \\ \mathbf{h}_{P}^{(l-1)} \end{array} \right] + \mathbf{b}^{(l)} \right)
\end{align*}

Tree-RNNは要素数 $T$ の計算グラフであっても分岐により深さ $\log_{2}(T)$ で済む。

自然言語処理では既存の解析器を用いて構文木を予測し、これをTree-RNNへ与えるものの  
構文解析は自明なタスクでなく、構文木が最適かどうかも不明である。